## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuatapere,NZ,-46.1333,167.6833,59.83,96,28,1.57,scattered clouds
1,1,Grindavik,IS,63.8424,-22.4338,32.47,69,75,17.27,broken clouds
2,2,Vitim,RU,59.4511,112.5578,-9.09,98,100,4.70,overcast clouds
3,3,Jaisalmer,IN,26.9147,70.9181,67.73,20,0,3.11,clear sky
4,4,Kapaa,US,22.0752,-159.3190,67.98,97,40,6.91,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Anito,PH,12.4497,125.2886,77.83,81,97,12.12,overcast clouds
8,8,Ponta Do Sol,PT,32.6667,-17.1000,71.71,79,100,6.08,overcast clouds
10,10,Cape Town,ZA,-33.9258,18.4232,75.61,52,20,10.36,few clouds
11,11,Manokwari,ID,-0.8667,134.0833,77.67,80,100,3.18,overcast clouds
12,12,Hermanus,ZA,-34.4187,19.2345,73.90,67,1,12.30,clear sky
18,18,Vaini,TO,-21.2000,-175.2000,80.28,82,80,23.42,light rain
20,20,Puerto Ayora,EC,-0.7393,-90.3518,80.56,84,92,5.01,overcast clouds
21,21,Phan Thiet,VN,10.9333,108.1000,75.49,81,63,9.84,broken clouds
23,23,Mount Isa,AU,-20.7333,139.5000,78.57,65,73,0.00,broken clouds
25,25,Atuona,PF,-9.8000,-139.0333,77.05,81,16,15.88,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                233
City                   233
Country                227
Lat                    233
Lng                    233
Max Temp               233
Humidity               233
Cloudiness             233
Wind Speed             233
Current Description    233
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Anito,PH,12.4497,125.2886,77.83,81,97,12.12,overcast clouds
8,8,Ponta Do Sol,PT,32.6667,-17.1000,71.71,79,100,6.08,overcast clouds
10,10,Cape Town,ZA,-33.9258,18.4232,75.61,52,20,10.36,few clouds
11,11,Manokwari,ID,-0.8667,134.0833,77.67,80,100,3.18,overcast clouds
12,12,Hermanus,ZA,-34.4187,19.2345,73.90,67,1,12.30,clear sky


In [7]:
#confirm Nan were dropped
clean_df.count()

City_ID                227
City                   227
Country                227
Lat                    227
Lng                    227
Max Temp               227
Humidity               227
Cloudiness             227
Wind Speed             227
Current Description    227
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Anito,PH,77.83,overcast clouds,12.4497,125.2886,
8,Ponta Do Sol,PT,71.71,overcast clouds,32.6667,-17.1000,
10,Cape Town,ZA,75.61,few clouds,-33.9258,18.4232,
11,Manokwari,ID,77.67,overcast clouds,-0.8667,134.0833,
12,Hermanus,ZA,73.90,clear sky,-34.4187,19.2345,
18,Vaini,TO,80.28,light rain,-21.2000,-175.2000,
20,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,
21,Phan Thiet,VN,75.49,broken clouds,10.9333,108.1000,
23,Mount Isa,AU,78.57,broken clouds,-20.7333,139.5000,
25,Atuona,PF,77.05,light rain,-9.8000,-139.0333,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Anito,PH,77.83,overcast clouds,12.4497,125.2886,FELIPA LAODENIO PROPERTY
8,Ponta Do Sol,PT,71.71,overcast clouds,32.6667,-17.1000,Hotel do Campo
10,Cape Town,ZA,75.61,few clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
11,Manokwari,ID,77.67,overcast clouds,-0.8667,134.0833,Swiss Belhotel Manokwari
12,Hermanus,ZA,73.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
18,Vaini,TO,80.28,light rain,-21.2000,-175.2000,Keleti Beach Resort
20,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Phan Thiet,VN,75.49,broken clouds,10.9333,108.1000,Hotel Saigon-PT
23,Mount Isa,AU,78.57,broken clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
25,Atuona,PF,77.05,light rain,-9.8000,-139.0333,Villa Enata


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [17]:
# confirm row count
clean_hotel_df.count()

City                   227
Country                227
Max Temp               227
Current Description    227
Lat                    227
Lng                    227
Hotel Name             227
dtype: int64

In [20]:
# 8a. Create the output File (CSV)
output_data_file = "../data/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}<dd/>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))